
# Laboratorium: Regresja wielowymiarowa — Wariant 10
**Temat:** Analiza zapotrzebowania na pracowników w firmach (wariant 10)  
**Cel:** Zbudować i ocenić modele sieci neuronowych przewidujące:  
- liczbę wymaganych pracowników  
- średnie wynagrodzenie  

Notebook zawiera: generację danych, preprocessing, porównanie 3 architektur sieci, wykresy i sprawozdanie.

> Dane treningowe (wg. instrukcji):  
> - cechy wejściowe: liczba projektów, średni czas trwania projektu (miesiące), budżet firmy  
> - wyjścia (target): liczba wymaganych pracowników, średnie wynagrodzenie  


In [ ]:

# Importy i ustawienia
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import matplotlib.pyplot as plt

# deterministyczność (tam gdzie to możliwe)
np.random.seed(42)
tf.random.set_seed(42)


## Generacja danych

In [ ]:

# Generacja danych zgodnie z Listing 13 (przeskalowane zgodnie z instrukcją)
N = 5000  # liczba próbek

# X: liczba projektów, średni czas trwania projektu (miesiące), budżet firmy
X = np.random.rand(N, 3) * np.array([50, 24, 10_000_000.0])

# y: liczba wymaganych pracowników, średnie wynagrodzenie
y = np.random.rand(N, 2) * np.array([500.0, 10_000.0])

# Uporządkowanie do DataFrame (dla czytelności)
df = pd.DataFrame(np.hstack([X, y]), columns=[
    'n_projects', 'avg_duration_months', 'budget',
    'req_employees', 'avg_salary'
])

df.head()


## Preprocessing (skalowanie, podział na zbiory)

In [ ]:

# Podział na zbiór treningowy i testowy oraz skalowanie cech
X = df[['n_projects', 'avg_duration_months', 'budget']].values
y = df[['req_employees', 'avg_salary']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja cech wejściowych (skalowanie budżetu ma duże wartości)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Standaryzacja targetów ułatwi trenowanie; zapiszemy skalery, aby odwrócić transformację
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


## Definicja architektur i funkcji pomocniczych

In [ ]:

# Funkcja pomocnicza do budowania modeli o różnych architekturach
def build_model(architecture='1hidden', input_shape=(3,)):
    model = keras.Sequential()
    if architecture == '1hidden':
        model.add(layers.Input(shape=input_shape))
        model.add(layers.Dense(64, activation='relu'))
    elif architecture == '2hidden':
        model.add(layers.Input(shape=input_shape))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(32, activation='relu'))
    elif architecture == '3hidden':
        model.add(layers.Input(shape=input_shape))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(32, activation='relu'))
    else:
        raise ValueError('Unknown architecture')
    model.add(layers.Dense(2))  # 2 wartości wyjściowe (regresja wielowymiarowa)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


## Trening modeli

In [ ]:

# Parametry treningu
EPOCHS = 200
BATCH_SIZE = 64
es = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

architectures = ['1hidden', '2hidden', '3hidden']
histories = {}
models = {}
results = {}

for arch in architectures:
    print('\n--- Trening modelu:', arch, '---')
    model = build_model(arch, input_shape=(X_train_scaled.shape[1],))
    history = model.fit(
        X_train_scaled, y_train_scaled,
        validation_split=0.15,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=0
    )
    models[arch] = model
    histories[arch] = history
    # ewaluacja na zestawie testowym (odwracamy skalowanie predykcji)
    y_pred_scaled = model.predict(X_test_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    results[arch] = {'mse': mse, 'mae': mae}
    print(f'Arch: {arch} -> MSE: {mse:.4f}, MAE: {mae:.4f}')


## Wyniki i wykresy

In [ ]:

# Wykresy: historia MSE (loss) dla każdej architektury
for arch in architectures:
    h = histories[arch]
    plt.figure()
    plt.plot(h.history['loss'], label='train_loss')
    plt.plot(h.history['val_loss'], label='val_loss')
    plt.title(f'Loss (MSE) - {arch}')
    plt.xlabel('Epoka')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid(True)
    plt.show()

# Scatter: rzeczywiste vs przewidywane (dla najlepszego modelu wg MSE)
best_arch = min(results, key=lambda k: results[k]['mse'])
print('\nNajlepsza architektura wg MSE:', best_arch, results[best_arch])

best_model = models[best_arch]
y_pred_best = scaler_y.inverse_transform(best_model.predict(X_test_scaled))

# Scatter dla obu wyjść w osobnych wykresach
plt.figure()
plt.scatter(y_test[:,0], y_pred_best[:,0], alpha=0.4)
plt.title('Rzeczywista vs Przewidywana - liczba wymaganych pracowników')
plt.xlabel('Rzeczywista liczba pracowników')
plt.ylabel('Przewidywana liczba pracowników')
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(y_test[:,1], y_pred_best[:,1], alpha=0.4)
plt.title('Rzeczywista vs Przewidywana - średnie wynagrodzenie')
plt.xlabel('Rzeczywista średnie wynagrodzenie')
plt.ylabel('Przewidywane średnie wynagrodzenie')
plt.grid(True)
plt.show()


## Tabela metryk

In [ ]:

# Tabela wyników metryk dla modeli
results_df = pd.DataFrame(results).T
results_df



---
# Sprawozdanie (podsumowanie eksperymentu)

**Opis zadania:** Wariant 10 — prognozowanie zapotrzebowania na pracowników i średniego wynagrodzenia na podstawie
liczby projektów, średniego czasu trwania projektu oraz budżetu firmy.

**Przyjęte założenia:** Dane wygenerowano syntetycznie zgodnie z Listing 13 z instrukcji laboratorium. Dane wejściowe i wyjściowe
zostały zeskalowane, a modele trenowano na znormalizowanych targetach (skalowanie odwrotne do oceny).

**Architektury testowane:**  
- `1hidden`: jedna warstwa ukryta 64 neurony  
- `2hidden`: dwie warstwy ukryte (64 → 32)  
- `3hidden`: trzy warstwy ukryte (128 → 64 → 32)

**Metryki oceny:** MSE i MAE na zbiorze testowym.  

**Wnioski:**  
- Porównując MSE i wykresy rzeczywiste vs przewidywane można ocenić, która architektura lepiej approximuje zależności.  
- Dla syntetycznych, losowo generowanych danych różnice mogą być niewielkie; w praktycznych zadaniach należy stosować regularizację, cross-validation oraz analizę wpływu cech (feature importance).

**Możliwe rozszerzenia:**  
- Zastosowanie K-Fold cross-validation, grid search (liczba warstw, liczba neuronów, learning rate).  
- Generowanie danych bardziej realistycznych (np. zależność budżetu od liczby projektów, korelacje między targetami).  
- Dodanie ograniczeń (np. wymagane pracownicy jako liczba całkowita — regresja z postprocessingiem do najbliższej liczby całkowitej).

---
